<h1 align=center><font size = 5>Applied Data Science Capstone - WEEK 3</font></h1> 
<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1> 
<h3><font size = 2>Coursera, Agosto 2019</font></h3> 

##### *Explore and cluster the neighborhoods in Toronto*
___



## Here begins question # 1. 
Built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

---

In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


In [4]:
# Read data from html to Panda Data Frame
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Toronto_DF = pd.read_html(url, header=0)[0]
#Data frame size
print('The table has',Toronto_DF.shape[0], 'rows and', Toronto_DF.shape[1], 'Columns')

#review data
Toronto_DF.head()

The table has 288 rows and 3 Columns


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# Ignore cells with a borough that is Not assigned
Toronto_DF['Borough']=Toronto_DF['Borough'].replace('Not assigned',np.nan)
Toronto_DF.dropna(axis=0, inplace= True) #Drop rows with 'Not assigned' as Borough
#New Data frame size
print('The table has',Toronto_DF.shape[0], 'rows and', Toronto_DF.shape[1], 'Columns')

#Check data Frame
Toronto_DF.head()


The table has 211 rows and 3 Columns


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [6]:
# Verify the quantity of unique Post Codes
print('There are',Toronto_DF['Postcode'].nunique(), 'unique Post Codes in Toronto')

There are 103 unique Post Codes in Toronto


In [7]:
# Group by 'Postcode' and wrangle the data, 
PostalGroups=Toronto_DF.groupby('Postcode')
Toronto_List=[] # Initialize a list
for name, group in PostalGroups:
    Neig_String=",".join(PostalGroups.get_group(name)['Neighbourhood'].tolist()) #combined Neighbourhoods in the same post code, into one row with the names separated with a comma
    Toronto_List.append ( [name,group.iloc[0,1],Neig_String] ) # Fill Toront_List

# Convert list into a Dataframe
column_names = ['Postcode','Borough','Neighborhood']
Toronto_Data=pd.DataFrame(Toronto_List,columns=column_names)
#New Data frame size
print('The table has',Toronto_Data.shape[0], 'rows and', Toronto_DF.shape[1], 'Columns')

Toronto_Data.head()


The table has 103 rows and 3 Columns


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
Toronto_Data.Neighborhood.loc[Toronto_Data.Neighborhood=='Not assigned']= Toronto_Data.Borough[Toronto_Data.Neighborhood=='Not assigned']

print('The table has',Toronto_Data.shape[0], 'rows and', Toronto_DF.shape[1], 'Columns')

Toronto_Data.head()

The table has 103 rows and 3 Columns


,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Here begins question # 2. 
Get the latitude and the longitude coordinates of each neighborhood.

---

In [9]:
import geocoder # import geocoder

In [10]:
Toronto_coords=[]

for postal_code in Toronto_Data['Postcode']:
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    Toronto_coords.append([lat_lng_coords[0],lat_lng_coords[1]])
print (len(Toronto_coords))

103


In [11]:
# Add Latitude and Longitude columns into Toronto_Data DF
column_names = ['Latitude','Longitude']
Toronto_Data_coord=Toronto_Data.join(pd.DataFrame(Toronto_coords,columns=column_names))
Toronto_Data_coord.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Here begins question # 3. 
Explore and cluster the neighborhoods in Toronto.

---

In [12]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [13]:
# Define Foursquare Constants: Credentials, Version and Limit
CLIENT_ID = '0WKQPIHLVV0E2XDXSZ20SZFW1OZR44GPZPIJDGZAAPMSH1NX' # your Foursquare ID
CLIENT_SECRET = 'FSM0E3XK0JMDT0PXBIZ21EEKNEDZU4XOFGBTSDRFUC5VQ0BF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0WKQPIHLVV0E2XDXSZ20SZFW1OZR44GPZPIJDGZAAPMSH1NX
CLIENT_SECRET:FSM0E3XK0JMDT0PXBIZ21EEKNEDZU4XOFGBTSDRFUC5VQ0BF


## Explore Neighborhoods in Toronto

In [14]:
# As in instrunctions: Work with only boroughs that contain the word Toronto 
Toronto_Data_filtered=Toronto_Data_coord[Toronto_Data_coord['Borough'].str.contains('Toronto')] 
Toronto_Data_filtered.shape

(38, 5)

#### Function to explore all the neighborhoods in Toronto

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the getNearbyVenues function on each neighborhood and create a new dataframe called *toronto_venues*.

In [16]:
toronto_venues = getNearbyVenues(names=Toronto_Data_filtered['Neighborhood'],
                                   latitudes=Toronto_Data_filtered['Latitude'],
                                   longitudes=Toronto_Data_filtered['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [17]:
# Check the size of toronto_venues DF
print('There are' ,toronto_venues.shape[0], 'venues')
toronto_venues.head()

There are 1750 venues


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.683262,-79.355120,Dairy Queen,43.684223,-79.357062,Fast Food Restaurant


In [18]:
# Count Venues per 'Neighborhood'
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,63,63,63,63,63,63
"Brockton,Exhibition Place,Parkdale Village",72,72,72,72,72,72
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",72,72,72,72,72,72
"Cabbagetown,St. James Town",42,42,42,42,42,42
Central Bay Street,98,98,98,98,98,98
"Chinatown,Grange Park,Kensington Market",98,98,98,98,98,98
Christie,10,10,10,10,10,10


#### Count unique categories from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 214 uniques categories.


## Analyze Each Neighborhood in Toronto

In [20]:
# Change the name for the Venue_Category called 'Neighborhood'
toronto_venues['Venue Category'].loc[toronto_venues['Venue Category']=='Neighborhood']= 'Neighbor'
# one hot encoding
Toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
print('There are', Toronto_onehot.shape[0], 'venues, into',  Toronto_onehot.shape[1], 'categories' )
Toronto_onehot.head()

There are 1750 venues, into 215 categories


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.015873,...,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.013889,0.000000,0.013889,0.000000,0.000000,0.013889,...,0.000000,0.000000,0.013889,0.027778,0.000000,0.000000,0.013889,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.020000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000,...,0.000000,0.013889,0.000000,0.013889,0.013889,0.000000,0.000000,0.000000,0.000000,0.013889
5,"Cabbagetown,St. James Town",0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.010204,0.000000,0.010204,0.010204,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010204,0.000000,0.010204,0.010204,0.010204,0.000000,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.010204,0.000000,0.020408,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.051020,0.000000,0.000000,0.040816,0.010204,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012346,0.012346,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.012346,0.000000,0.000000,0.000000


#### Print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.07
2                Hotel  0.05
3  American Restaurant  0.03
4           Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2          Restaurant  0.05
3               Hotel  0.03
4  Seafood Restaurant  0.03


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop  0.10
1                    Café  0.06
2  Furniture / Home Store  0.06
3              Restaurant  0.06
4          Sandwich Place  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.09
1          Bar  0.05
2         Café  0.05
3   Steakhouse  0.04
4        Hotel  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0         Coffee Shop  0.11
1  Ital

#### Convert top 5 into a *pandas* dataframe

In [23]:
# Function to sort freq of venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Deli / Bodega,Asian Restaurant,Steakhouse,Restaurant,Bar,Burger Joint,Gastropub
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Breakfast Spot,Steakhouse,Hotel,Seafood Restaurant,Beer Bar,Farmers Market,Cheese Shop
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Furniture / Home Store,Restaurant,Café,Sandwich Place,Bar,Hotel,Vegetarian / Vegan Restaurant,Italian Restaurant,Supermarket
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Café,Hotel,Steakhouse,Italian Restaurant,American Restaurant,Thai Restaurant,Pizza Place,Japanese Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Café,Restaurant,Bar,Bakery,Park,Speakeasy,Sandwich Place,Pub


## Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 2,
       4, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_Data_filtered

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676845,-79.295225,0.0,Health Food Store,Pub,Trail,Neighbor,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
41,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120,2.0,Bus Line,Park,Discount Store,Fast Food Restaurant,Grocery Store,Elementary School,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314673,0.0,Park,Sandwich Place,Burrito Place,Pub,Burger Joint,Food & Drink Shop,Liquor Store,Steakhouse,Sushi Restaurant,Coffee Shop
43,M4M,East Toronto,Studio District,43.662766,-79.334830,0.0,Pizza Place,Diner,Italian Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Café,Bar,Brewery,Arts & Crafts Store
44,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,4.0,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [32]:
# get the geographical coordinates of Toronto.
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
Toronto_merged.dropna(axis=0, inplace= True) #Drop rows with No venues found
#Toronto_merged['Cluster Labels'].astype('int32')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters
Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

### CLUSTER 1
Coffee Shops

In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0.0,Health Food Store,Pub,Trail,Neighbor,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
42,East Toronto,0.0,Park,Sandwich Place,Burrito Place,Pub,Burger Joint,Food & Drink Shop,Liquor Store,Steakhouse,Sushi Restaurant,Coffee Shop
43,East Toronto,0.0,Pizza Place,Diner,Italian Restaurant,Bakery,Coffee Shop,Sushi Restaurant,Café,Bar,Brewery,Arts & Crafts Store
45,Central Toronto,0.0,Hotel,Park,Breakfast Spot,Gym,Food & Drink Shop,Clothing Store,Yoga Studio,Event Space,Flower Shop,Flea Market
47,Central Toronto,0.0,Dessert Shop,Italian Restaurant,Sandwich Place,Coffee Shop,Pizza Place,Café,Park,Indoor Play Area,Restaurant,Seafood Restaurant
48,Central Toronto,0.0,Playground,Tennis Court,Gym,Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
49,Central Toronto,0.0,Light Rail Station,Coffee Shop,Supermarket,Liquor Store,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
51,Downtown Toronto,0.0,Coffee Shop,Café,Italian Restaurant,Restaurant,Pizza Place,Bakery,Park,Pharmacy,Deli / Bodega,Chinese Restaurant
52,Downtown Toronto,0.0,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Bubble Tea Shop,Pub,Café,Pizza Place,Dance Studio
53,Downtown Toronto,0.0,Coffee Shop,Breakfast Spot,Italian Restaurant,Mexican Restaurant,Spa,Gym / Fitness Center,Restaurant,Electronics Store,Food Truck,Event Space


### CLUSTER 2

In [41]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,1.0,Harbor / Marina,Café,Music Venue,Yoga Studio,Elementary School,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


### CLUSTER 3
Parks and Fish Markers

In [38]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,2.0,Bus Line,Park,Discount Store,Fast Food Restaurant,Grocery Store,Elementary School,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
46,Central Toronto,2.0,Playground,Gym Pool,Park,Garden,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
50,Downtown Toronto,2.0,Playground,Building,Park,Bank,Elementary School,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
82,West Toronto,2.0,Convenience Store,Park,Intersection,Sandwich Place,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


### CLUSTER 4

In [39]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Central Toronto,3.0,Park,Yoga Studio,Elementary School,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


### CLUSTER 5

In [40]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4.0,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
